In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [4]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [6]:
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'

In [8]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [14]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [15]:
data_cleaning(first_file)
data_cleaning(second_file)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2020-07-24
2020-07-25
2020-07-22
2020-07-23


In [16]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                     2020-07-24            2020-07-25            2020-07-22  \
Product                      HS        MS          HS         MS         HS   
SA                                                                            
Fuels- Bargarh(R)      87204.30  43207.67   93192.590  41821.580  100914.49   
Fuels-Bhawanipatna(R   96424.20  48369.35   77081.120  33702.860   94789.20   
Fuels-JEYPORE(R)      134895.26  67461.87  100569.200  47596.350  149491.60   
Fuels-Jharsuguda(R)    97742.51  30288.87   89493.180  26049.733  136289.78   
Fuels-ROURKELA(R)     316276.66  36147.24  322542.880  35294.500  290886.78   
Fuels-SAMBALPUR (R)   120257.52  23414.59  116704.310  20810.150  140052.31   
Keonjhar I RSA         87804.07  15939.81  113298.817  14346.960  117875.81   
Keonjhar II RSA        12213.20   6336.09   13601.470   4985.270   21413.61   

                                2020-07-23            
Product                     MS          HS        MS  
SA                  

In [17]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [18]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-07-24' 'Unnamed: 2' '2020-07-25' 'Unnamed: 4'
 '2020-07-22' 'Unnamed: 6' '2020-07-23' 'Unnamed: 8']


In [19]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-07-25', 'Unnamed: 4', '2020-07-24', 'Unnamed: 2', '2020-07-23', 'Unnamed: 8', '2020-07-22', 'Unnamed: 6']


In [20]:
df = df[cols]
print(df)

             Unnamed: 0 2020-07-25 Unnamed: 4 2020-07-24 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    93192.6    41821.6    87204.3    43207.7   
3  Fuels-Bhawanipatna(R    77081.1    33702.9    96424.2    48369.3   
4      Fuels-JEYPORE(R)     100569    47596.3     134895    67461.9   
5   Fuels-Jharsuguda(R)    89493.2    26049.7    97742.5    30288.9   
6     Fuels-ROURKELA(R)     322543    35294.5     316277    36147.2   
7   Fuels-SAMBALPUR (R)     116704    20810.2     120258    23414.6   
8        Keonjhar I RSA     113299      14347    87804.1    15939.8   
9       Keonjhar II RSA    13601.5    4985.27    12213.2    6336.09   

  2020-07-23 Unnamed: 8 2020-07-22 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    95697.2    49825.2     100914    51743.5  
3    98421.6    49654.7   

In [22]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [25]:
wb = load_workbook(first_file)
ws = wb['summary']
print(ws['A1'].value)

Unnamed: 0


In [36]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"